In [1]:
load_ext cython

In [2]:
%%cython

cpdef pack_retina(int offset, int kernel_idx, double[::1] values, int size, unsigned short[::1] xs, unsigned short[::1] ys, loc, double[::1] coeff_array, index_array, unsigned int[:,::1] kernel_map):
    cdef unsigned int idx = 0
    cdef unsigned int kernel_size, i, j, k
    cdef unsigned long long pixel_location
    cdef unsigned short x1, y1
    
    kernel_size = loc[6]
    x1 = loc[0] - kernel_size/2+0.5
    y1 = loc[1] - kernel_size/2+0.5
    for j in range(kernel_size):
        for i in range(kernel_size):
            pixel_location = ((ys[j]+y1)*size)+(xs[i]+x1)
            coeff_array[offset+idx] = values[idx]
            index_array[offset+idx] = pixel_location
            for k in range(kernel_map.shape[0]):
                if kernel_map[k][pixel_location] == 0:
                    kernel_map[k][pixel_location] = kernel_idx+1
                    break
            idx+=1

cpdef pack_cortex(int offset, int kernel_idx, double[::1] values, int size, unsigned short[::1] xs, unsigned short[::1] ys, loc, double[::1] coeff_array, index_array):
    cdef unsigned int idx = 0
    cdef unsigned int kernel_size, i, j
    cdef unsigned long long pixel_location
    cdef unsigned short x1, y1
    
    kernel_size = loc[6]
    x1 = loc[0] - kernel_size/2+0.5
    y1 = loc[1] - kernel_size/2+0.5
    for j in range(kernel_size):
        for i in range(kernel_size):
            pixel_location = ((ys[j]+y1)*size)+(xs[i]+x1)
            coeff_array[offset+idx] = values[idx]
            index_array[offset+idx] = pixel_location
            idx+=1

In [3]:
import numpy as np
import tkinter as tk
from tkinter import Tk, ttk
from tkinter import filedialog as fd
from tkinter import messagebox as mb
from helpers import *
import subprocess as proc
from threading import Thread

import time

types = {'INPUT': None,
         'COEFFICIENT': None,
         'RESULT': None,
         'BACK_PROJECTED': None,
         'Kernel_Size' : None,
         'INDEX': None
        }

cython_types = {
    'uint8' : 'unsigned char',
    'uint16' : 'unsigned short',
    'uint32' : 'unsigned int',
    'uint64' : 'unsigned long long',
    'float32' : 'float',
    'float64' : 'double'}

threadStatus = {'active_threads': [],
                'manager': True,
                'compile': False,
                'retina': False,
                'cortex':False}

bit_ranges = ((0,8),(9,16),(17,32),(33,64))

verified_values = {"image":8, "coeff":16}

quantize = True

class layerGenerator:
    def __init__(self, mode):
        
        self.size = None
        self.locs = None
        self.coeffs = None
        self.scalingFactor = None
        self.quantization_bits = None
        
        if 'retina' in mode.lower():
            self.mode = 'retina'
        elif 'cortex' in mode.lower():
            self.mode = 'cortex'
        else:
            print("Invalid packing mode selected!")

    def calcSize(self):
        if self.size is None:
            if self.mode == 'retina':
                w = 2*np.round(np.abs(self.locs[:,0]).max() + self.locs[:,6].max()/2.0)
                h = 2*np.round(np.abs(self.locs[:,1]).max() + self.locs[:,6].max()/2.0)
                self.locs[:,:2] = (self.locs[:,:2]+ np.array((w//2,h//2)))
            else:
                w = np.round(self.locs[:,0].max() + self.locs[:,6].max()/2.0)
                h = np.round(self.locs[:,1].max() + self.locs[:,6].max()/2.0)
            self.size = np.array([h,w], dtype='int32')

    def pack(self, thread, toInts=True, progBar={"value":0}):
        ######################################
        ########### INITIALIZATION ###########
        ######################################
        h, w = self.size
        
        total_elements = int((self.locs[:,6]*self.locs[:,6]).sum())

        self.coeff_array = np.zeros(total_elements, dtype='float64')
        self.index_array = np.zeros(total_elements, dtype=types['INDEX'])
        self.size_array = np.zeros(len(self.locs), dtype=types['Kernel_Size'])
        self.kernel_map = []
        #######################################
        ###### FINDING OVERLAPPING AREAS ######
        #######################################
        if self.mode == 'retina':
            overlapCounter = np.zeros((h,w),dtype='uint8')
            for kernel in range(len(self.coeffs)):
                kernel_size = self.locs[kernel][6]
                y1 = int(self.locs[:,1][kernel] - kernel_size/2+0.5)
                y2 = int(self.locs[:,1][kernel] + kernel_size/2+0.5)
                x1 = int(self.locs[:,0][kernel] - kernel_size/2+0.5)
                x2 = int(self.locs[:,0][kernel] + kernel_size/2+0.5)
                overlapCounter[y1:y2,x1:x2] += 1
            nLayers = overlapCounter.max()
            [self.kernel_map.append(np.zeros((h*w))) for _ in range(nLayers)]
            
            self.kernel_map = np.array(self.kernel_map, dtype='uint32')
        #######################################
        ######### Generating Arrays ###########
        #######################################
        max_kernel_size = self.locs[:,6].max()
        xs = np.arange(max_kernel_size, dtype='uint16')
        ys = np.arange(max_kernel_size, dtype='uint16')
        kernel = 0
        offset = 0
        if self.mode == 'retina':
            while kernel < len(self.coeffs) and threadStatus[thread]:
                values = self.coeffs[kernel].ravel()
                pack_retina(offset, kernel,  values, self.size[1], xs, ys, self.locs[kernel], self.coeff_array, self.index_array, self.kernel_map)
                self.size_array[kernel] = self.locs[kernel,6]**2
                offset += len(values)
                kernel+=1
                
                if kernel%100==0:
                    progBar["value"]= 100.0*kernel/len(self.coeffs)
        else:
            while kernel < len(self.coeffs) and threadStatus[thread]:
                values = self.coeffs[kernel].ravel()
                pack_cortex(offset, kernel,  values, self.size[1], xs, ys, self.locs[kernel], self.coeff_array, self.index_array)
                self.size_array[kernel] = self.locs[kernel,6]**2
                offset += len(values)
                kernel+=1
                
                if kernel%100==0:
                    progBar["value"]= 100.0*kernel/len(self.coeffs)
        
        if not threadStatus[thread]:
            return
        #######################################
        ###### CONVERTING FLOATS TO INTS ######
        #######################################
        
        if toInts:
            self.scalingFactor = ((2**self.quantization_bits)-1) / np.max(self.coeff_array)
            self.coeff_array = np.round(self.coeff_array*self.scalingFactor)
        else:
            self.scalingFactor = 1
        self.coeff_array = self.coeff_array.astype(types['COEFFICIENT'])
        
        

In [4]:
def threadManager():
    while threadStatus['manager']:
        [t.join() for t in threadStatus['active_threads'] if not t.is_alive()]
        threadStatus['active_threads'] = [t for t in threadStatus['active_threads'] if t.is_alive()]
        time.sleep(0.1)
    print("Exiting manager")

def correctedBits(x):
    for i in range(len(bit_ranges)):
        if bit_ranges[i][0] <= x <= bit_ranges[i][1]:
            return bit_ranges[i][1]
        
def generateConfig():
    types['INPUT'] = "uint"+str(correctedBits(slider1.get()))
    if quantize:
        types['COEFFICIENT'] = "uint"+str(correctedBits(slider2.get()))
        types['RESULT'] = "uint"+str(correctedBits(slider3.get()))
        types['BACK_PROJECTED'] = "uint"+str(correctedBits(slider4.get()))
    types['Kernel_Size'] = "uint"+str(correctedBits(slider5.get()))
    types['INDEX'] = "uint"+str(correctedBits(slider6.get()))
    try:
        savePickle("config.pkl",types)
        printOutput("Configuration File generated succesfully.")
    except:
        printOutput("Something went wrong while saving the output file.")
    printOutput(spacer)

def loadConfig():
    global types
    try:
        types = loadPickle("config.pkl")
        return 0
    except:
        printOutput("Failed to load config.pkl, please generate a config file first.")
        printOutput(spacer)
        return -1
    
def generateCython():
    output=[]
    if loadConfig() != 0:
        return
    
    try:
        with open('cython_template.py', 'r') as f:
            i = 1
            for line in f:
                code = line
                if 'def' in code:
                    if (code.count(',')+1) != code.count('{'):
                        printOutput("Warning: Unwrapped type detected at line %i"%i)
                        printOutput(code)
                        printOutput(spacer)
                for typ in types:
                    indicator = '{%s}'%typ
                    if indicator in code:
                        code = code.replace(indicator,cython_types[types[typ]])
                output.append(code)
                i+=1
    except:
        printOutput("Failed to load cython_template.py, please make sure it exists.")
        printOutput(spacer)
        return

    try:
        with open('functions.pyx', 'w') as f:
            f.write(''.join(output))
                
        printOutput("Generated extension Succesfully.")
    except:
        printOutput("Something went wrong while saving the output file.")
    printOutput(spacer)

def compileCython():
    button3.configure(state='disabled')
    printOutput("Compiling ...")
    printOutput(spacer)
    p1 = proc.run('python setup.py build_ext --inplace', shell=True, capture_output=True, text=True)
    if p1.returncode == 0:
        printOutput(p1.stdout)
        printOutput(spacer)
        printOutput("Compilation finished successfully!")
        printOutput(spacer)
        printOutput("Generating report ...")
        p2 = proc.run('cython -a functions.pyx', shell=True, capture_output=True, text=True)
        if p2.returncode == 0:
            printOutput("Reprot generated successfully!")
        else:
            printOutput(p2.stderr)
    else:
        printOutput(p1.stderr)
    printOutput(spacer)
    button3.configure(state='normal')
    threadStatus['compile'] = False

def loadLoc(generator, lbl_dir):
    failed = False
    try:
        fname = fd.askopenfilename(title = "Select locations data", filetypes = (("Pickle Object","*.pkl"),
                                                                                 ("all files","*.*")))
        if not fname:
            return
        with open(fname, "rb" ) as f:
            generator.locs = pickle.load(f, encoding='latin1')
        if generator.locs.shape[-1] != 7:
            mb.showerror("Failed!", "Data has wrong dimensions.\nPlease make sure you have selected the correct file",
                         parent=tab2)
            failed = True
        else:
            lbl_dir.config(text=fname)
            return 0
    except:
        printOutput("Something went wrong while trying to load locations.")
        printOutput("Please make sure you have selected the correct file and that the file is not in use by aother process.")
        printOutput(spacer)
        return 1
    
    if failed:
        try:
            with open(lbl_dir["text"], "rb" ) as f:
                generator.locs = pickle.load(f, encoding='latin1')
            return 0
        except:
            generator.locs = None
            lbl_dir.config(text="None")
            return 0
            
def loadCoeff(generator, lbl_dir):
    failed = False
    try:
        fname = fd.askopenfilename(title = "Select coefficients data", filetypes = (("Pickle Object","*.pkl"),
                                                                                    ("all files","*.*")))
        if not fname:
            return
        with open(fname, "rb" ) as f:
            generator.coeffs = np.squeeze(pickle.load(f, encoding='latin1'))
        if len(generator.coeffs.shape) != 1 or len(generator.locs) != len(generator.coeffs):
            mb.showerror("Failed!", "Data has wrong dimensions.\nPlease make sure you have selected the correct file",
                         parent=tab2)
            failed = True
        else:
            lbl_dir.config(text=fname)
            return 0
    except:
        printOutput("Something went wrong while trying to load coefficients.")
        printOutput("Please make sure you have selected the correct file and that the file is not in use by aother process.")
        printOutput(spacer)
        return 1
    
    if failed:
        try:
            with open(lbl_dir["text"], "rb" ) as f:
                generator.coeffs = pickle.load(f, encoding='latin1')
            return 0
        except:
            generator.coeffs = None
            lbl_dir.config(text="None")
            return 1

def generateArrays(generator, btn, entry, progBar, cmd,thread):
    if loadConfig() != 0:
        return
    printOutput("Generating arrays ...")
    invalids = " %:/,.\\[]<>*?"
    outputName = ''.join([c for c in entry.get() if c not in invalids])+'.pkl'
    generator.quantization_bits = int(''.join([c for c in types['COEFFICIENT'] if c.isdigit()]))
    generator.calcSize()
    generator.pack(thread, toInts='int' in types['COEFFICIENT'], progBar=progBar)
    if not threadStatus[thread]:
        printOutput("Operation aborted")
        printOutput(spacer)
        return
    if generator.mode == 'cortex':
        generator.kernel_map = np.array((left_hemi_generator.size, right_hemi_generator.size))
    printOutput("Saving to output file ...")
    try:
        savePickle(outputName,(generator.size,
                               generator.scalingFactor,
                               generator.coeff_array,
                               generator.index_array,
                               generator.size_array,
                               generator.kernel_map))
        printOutput("Successfully saved to %s" %outputName)
    except:
        printOutput("Something went wrong while saving the output file.")
    printOutput(spacer)
    threadStatus[thread] = False
    btn.config(text="Generate arrays", command=cmd)

def on_closing():
    threadStatus['manager'] = False
    root.destroy()
    return
    if mb.askokcancel("Quit", "Do you want to quit?"):
        root.destroy()

def printOutput(text):
    output.configure(state='normal')
    output.insert('end', str(text)+'\n')
    output.configure(state='disabled')
    output.see("end")
    
def clearOutput():
    output.configure(state='normal')
    output.delete("1.0","end")
    output.configure(state='disabled')
    
def combo1Change(x):
    global quantize
    val = combo1.get().lower()
    if 'int' in val:
        quantize = True
    elif '32' in val:
        quantize = False
        types['COEFFICIENT'] = 'float32'
        types['RESULT'] = 'float32'
        types['BACK_PROJECTED'] = 'float32'
    elif '64' in val:
        quantize = False
        types['COEFFICIENT'] = 'float64'
        types['RESULT'] = 'float64'
        types['BACK_PROJECTED'] = 'float64'
    tab1.focus()

def slider1Change(x):
    val = int(slider1.get())
    resultVal = int(slider2.get())+val+1
    backProjectedVal = int(slider2.get())+resultVal+1
    if backProjectedVal<=64:
        label8.config(text=val)
        slider1.config(value = val)
        slider3.config(value = resultVal)
        label10.config(text=resultVal)
        slider4.config(value = backProjectedVal)
        label11.config(text=backProjectedVal)
        verified_values["image"] = val
    else:
        val = verified_values["image"]
        label8.config(text=val)
        slider1.config(value = val)
        

def slider2Change(x):
    val = int(slider2.get())
    resultVal = int(slider1.get())+val+1
    backProjectedVal = val+resultVal+1
    if backProjectedVal<=64:
        label9.config(text=val)
        slider2.config(value = val)
        slider3.config(value = resultVal)
        label10.config(text=resultVal)
        slider4.config(value = backProjectedVal)
        label11.config(text=backProjectedVal)
        verified_values["coeff"] = val
    else:
        val = verified_values["coeff"]
        label9.config(text=val)
        slider2.config(value = val)
    
def slider5Change(x):
    val = int(slider5.get())
    label12.config(text=val)
    slider5.config(value = val)
    
def slider6Change(x):
    val = int(slider6.get())
    label13.config(text=val)
    slider6.config(value = val)

def compile_T():
    t = Thread(target=compileCython)
    t.start()
    threadStatus['compile'] = True
    threadStatus['active_threads'].append(t)

def browseRetinaLoc():
    if loadLoc(retina_generator, label15)==0:
        button5.configure(state='normal')
    else:
        button5.configure(state='disabled')

def browseRetinaCoeff():
    if loadCoeff(retina_generator, label17)==0:
        button6.configure(state='normal')
    else:
        button6.configure(state='disabled')

def generateRet():
    t = Thread(target=generateArrays, args=(retina_generator, button6, retinaFileName, retinaProgBar, generateRet, 'retina'))
    t.start()
    threadStatus['retina'] = True
    threadStatus['active_threads'].append(t)
    button6.config(text="Abort", command=abortRet)
    
def abortRet():
    threadStatus['retina'] = False
    button6.config(text="Generate arrays", command=generateRet)

def browseLeftHemiLoc():
    if loadLoc(left_hemi_generator, label21)==0:
        button8.configure(state='normal')
    else:
        button8.configure(state='disabled')

def browseLeftHemiCoeff():
    if loadCoeff(left_hemi_generator, label23)==0 and right_hemi_generator.coeffs is not None:
        button9.configure(state='normal')
    else:
        button9.configure(state='disabled')

def browseRightHemiLoc():
    if loadLoc(right_hemi_generator, label25)==0:
        button11.configure(state='normal')
    else:
        button11.configure(state='disabled')

def browseRightHemiCoeff():
    if loadCoeff(right_hemi_generator, label27)==0 and left_hemi_generator.coeffs is not None:
        button9.configure(state='normal')
    else:
        button9.configure(state='disabled')

def generateCortex():
    left_hemi_generator.calcSize()
    y_offset = left_hemi_generator.size[0]
    right_hemi_generator.calcSize()
    right_hemi_generator.locs[:,1] += y_offset
    cortex_generator.size = np.array([left_hemi_generator.size[0]+right_hemi_generator.size[0], max(left_hemi_generator.size[1], right_hemi_generator.size[1])], dtype='int32')

    cortex_generator.locs = np.concatenate((left_hemi_generator.locs, right_hemi_generator.locs))
    cortex_generator.coeffs = np.concatenate((left_hemi_generator.coeffs, right_hemi_generator.coeffs))
    order = np.argsort(cortex_generator.locs[:,2])
    cortex_generator.locs = cortex_generator.locs[order]
    cortex_generator.coeffs = cortex_generator.coeffs[order]
    
    t = Thread(target=generateArrays, args=(cortex_generator, button9, cortexFileName, cortexProgBar, generateCortex, 'cortex'))
    t.start()
    threadStatus['cortex'] = True
    threadStatus['active_threads'].append(t)
    button9.config(text="Abort", command=abortCortex)

def abortCortex():
    threadStatus['cortex'] = False
    button9.config(text="Generate arrays", command=generateCortex)

####################################
############### INIT ###############
####################################

retina_generator = layerGenerator('retina')
left_hemi_generator = layerGenerator('cortex')
right_hemi_generator = layerGenerator('cortex')
cortex_generator = layerGenerator('cortex')

spacer = "="*79

####################################
############### ROOT ###############
####################################

root = Tk()
wScreen, hScreen = root.winfo_screenwidth(), root.winfo_screenheight()
root.lift()
root.title("Retina Generator")
root.geometry('640x480')
root.resizable(False, False)

tabControl1 = ttk.Notebook(root)
  
tab1 = ttk.Frame(tabControl1, width=640, height =265)
tab2 = ttk.Frame(tabControl1, width=640, height =265)
tab3 = ttk.Frame(tabControl1, width=640, height =265)
  
tabControl1.add(tab1, text ='Configuration')
tabControl1.add(tab2, text ='Retina Transformer')
tabControl1.add(tab3, text ='Cortex Transformer')

tabControl2 = ttk.Notebook(tab3)

left_hemi = ttk.Frame(tabControl2, height =150)
right_hemi = ttk.Frame(tabControl2, height =150)

tabControl2.add(left_hemi, text ='Left hemisphere')
tabControl2.add(right_hemi, text ='Right hemisphere')

#####################################
############### TAB 1 ###############
#####################################
label1 = ttk.Label(tab1, text="Please select the number of bits used for each variable")
label28 = ttk.Label(tab1, text="Type of coefficients:")
label2 = ttk.Label(tab1, text="Image")
label3 = ttk.Label(tab1, text="Coeffients")
label4 = ttk.Label(tab1, text="Sampling result")
label5 = ttk.Label(tab1, text="Back projection")
label6 = ttk.Label(tab1, text="Kernel Size")
label7 = ttk.Label(tab1, text="Pixel index")
label8 = ttk.Label(tab1, text="8")
label9 = ttk.Label(tab1, text="16")
label10 = ttk.Label(tab1, text="25")
label11 = ttk.Label(tab1, text="42")
label12 = ttk.Label(tab1, text="16")
label13 = ttk.Label(tab1, text="32")

slider1 = ttk.Scale(tab1, from_=8, to=16, orient=tk.HORIZONTAL, value=8, command=slider1Change)
slider2 = ttk.Scale(tab1, from_=8, to=27, orient=tk.HORIZONTAL, value=16, command=slider2Change)
slider3 = ttk.Scale(tab1, from_=17, to=40, orient=tk.HORIZONTAL, value=25)
slider3.configure(state='disabled')
slider4 = ttk.Scale(tab1, from_=26, to=64, orient=tk.HORIZONTAL, value=42)
slider4.configure(state='disabled')
slider5 = ttk.Scale(tab1, from_=8, to=32, orient=tk.HORIZONTAL, value=16, command=slider5Change)
slider6 = ttk.Scale(tab1, from_=32, to=64, orient=tk.HORIZONTAL, value=32, command=slider6Change)

button1 = ttk.Button(tab1,text="Generate configuration file", command=generateConfig)
button2 = ttk.Button(tab1,text="Convert cython template to cython extension", command=generateCython)
button3 = ttk.Button(tab1,text="Compile cython extension", command=compile_T)

combo1 = ttk.Combobox(tab1, value=['Integer', 'Float64', 'Float32'], state='readonly', width=7)
combo1.current(0)
combo1.bind("<<ComboboxSelected>>", combo1Change)

offset = 60
height = 30
xOffset = 100
labelOffset = 120

label1.place(x=0,y=0)
label28.place(x=0,y=30)
label2.place(x=0,y=offset)
label3.place(x=0,y=offset+(height))
label4.place(x=0,y=offset+(2*height))
label5.place(x=0,y=offset+(3*height))
label6.place(x=0,y=offset+(4*height))
label7.place(x=0,y=offset+(5*height))

slider1.place(x=xOffset,y=offset)
slider2.place(x=xOffset,y=offset+(height))
slider3.place(x=xOffset,y=offset+(2*height))
slider4.place(x=xOffset,y=offset+(3*height))
slider5.place(x=xOffset,y=offset+(4*height))
slider6.place(x=xOffset,y=offset+(5*height))

label8.place(x=xOffset+labelOffset,y=offset)
label9.place(x=xOffset+labelOffset,y=offset+(height))
label10.place(x=xOffset+labelOffset,y=offset+(2*height))
label11.place(x=xOffset+labelOffset,y=offset+(3*height))
label12.place(x=xOffset+labelOffset,y=offset+(4*height))
label13.place(x=xOffset+labelOffset,y=offset+(5*height))

button1.place(x=0,y=offset+(6*height))
button2.place(x=160,y=offset+(6*height))
button3.place(x=420,y=offset+(6*height))

combo1.place(x=110,y = 30)

#####################################
############### TAB 2 ###############
#####################################
tab_offset = 23
x_offset = 1
lbl_height = 20
row_offset = 80

label14 = ttk.Label(tab2, text="Retina locations file")
label15 = ttk.Label(tab2, text="None")
label16 = ttk.Label(tab2, text="Retina coefficients file")
label17 = ttk.Label(tab2, text="None")
label18 = ttk.Label(tab2, text="Output file name :")
label19 = ttk.Label(tab2, text=".pkl")

button4 = ttk.Button(tab2,text="Browse ...", command=browseRetinaLoc)
button5 = ttk.Button(tab2,text="Browse ...", command=browseRetinaCoeff)
button5.config(state="disabled")
button6 = ttk.Button(tab2,text="Generate arrays", command=generateRet)
button6.config(state="disabled")

retinaFileName = ttk.Entry(tab2)
retinaFileName.insert(0,"retina")

retinaProgBar = ttk.Progressbar(tab2, orient=tk.HORIZONTAL, length=635, mode="determinate")

label14.place(x=x_offset,y=tab_offset)
label15.place(x=x_offset,y=tab_offset+lbl_height)
label16.place(x=x_offset,y=tab_offset+row_offset)
label17.place(x=x_offset,y=tab_offset+row_offset+lbl_height)
label18.place(x=x_offset,y=178)
label19.place(x=x_offset+230,y=178)


button4.place(x=x_offset,y=tab_offset+2*lbl_height)
button5.place(x=x_offset,y=tab_offset+row_offset+(2*lbl_height))
button6.place(x=x_offset,y=233)

retinaFileName.place(x=x_offset+105,y=178)

retinaProgBar.place(x=x_offset,y=208)

#####################################
############### TAB 3 ###############
#####################################

label24 = ttk.Label(tab3, text="Output file name :")
label25 = ttk.Label(tab3, text=".pkl")

button9 = ttk.Button(tab3,text="Generate arrays", command=Thread(target=generateCortex).start)
button9.config(state="disabled")

cortexFileName = ttk.Entry(tab3)
cortexFileName.insert(0,"cortex")

cortexProgBar = ttk.Progressbar(tab3, orient=tk.HORIZONTAL, length=635, mode="determinate")

label24.place(x=0,y=180)
label25.place(x=230,y=180)

button9.place(x=0,y=235)

cortexFileName.place(x=105,y=180)

cortexProgBar.place(x=0,y=210)

###################
#### LEFT HEMI ####
###################

label20 = ttk.Label(left_hemi, text="Retina locations file")
label21 = ttk.Label(left_hemi, text="None")
label22 = ttk.Label(left_hemi, text="Retina coefficients file")
label23 = ttk.Label(left_hemi, text="None")

button7 = ttk.Button(left_hemi,text="Browse ...", command=browseLeftHemiLoc)
button8 = ttk.Button(left_hemi,text="Browse ...", command=browseLeftHemiCoeff)
button8.config(state="disabled")

label20.place(x=0,y=0)
label21.place(x=0,y=lbl_height)
label22.place(x=0,y=row_offset)
label23.place(x=0,y=row_offset+lbl_height)

button7.place(x=0,y=2*lbl_height)
button8.place(x=0,y=row_offset+(2*lbl_height))

####################
#### RIGHT HEMI ####
####################

label24 = ttk.Label(right_hemi, text="Retina locations file")
label25 = ttk.Label(right_hemi, text="None")
label26 = ttk.Label(right_hemi, text="Retina coefficients file")
label27 = ttk.Label(right_hemi, text="None")
label28 = ttk.Label(right_hemi, text="Output file name :")
label29 = ttk.Label(right_hemi, text=".pkl")

button10 = ttk.Button(right_hemi,text="Browse ...", command=browseRightHemiLoc)
button11 = ttk.Button(right_hemi,text="Browse ...", command=browseRightHemiCoeff)
button11.config(state="disabled")

label24.place(x=0,y=0)
label25.place(x=0,y=lbl_height)
label26.place(x=0,y=row_offset)
label27.place(x=0,y=row_offset+lbl_height)

button10.place(x=0,y=2*lbl_height)
button11.place(x=0,y=row_offset+(2*lbl_height))

####################################
############### ROOT ###############
####################################

tabControl1.place(x=0,y=0)
tabControl2.pack(side='top', fill='x')

output = tk.Text(root,width=91, height=10, font=("TkDefaultFont",10))
output.place(x=0,y=291)
output.configure(state='disabled')

buttonClr = ttk.Button(root,text="Clear output", width=105, command=clearOutput)
buttonClr.place(x=0,y=455)

root.protocol("WM_DELETE_WINDOW", on_closing)
manager = Thread(target=threadManager)
manager.start()

root.mainloop()

Exiting manager
